In [1]:
import sys
sys.path.append('./../')

In [2]:
import ikamoana.fisherieseffort.fisherieseffort as fe
from ikamoana.ikamoanafields import ikamoanafields

## Initialize

In [3]:
xml_filepath="./../data/feeding_habitat_data/po_2x30_historical_interim/skj_po_interim_2deg_configuration_file.xml"
fisheries_xml_filepath="./../data/feeding_habitat_data/po_2x30_historical_interim/ORIGINAL_skj_interim_CLT_2018_testopt.xml"

ikaf = ikamoanafields.IkamoanaFields(xml_filepath,xml_filepath)
ikaf.feeding_habitat_structure.data_structure.normalizeCoords()

# Compute Taxis

In [4]:
taxis = ikaf.computeEvolvingTaxis()

# Compute Diffusion

In [5]:
diffusion = ikaf.diffusion(ikaf.feeding_habitat)

# Compute Mortality

In [6]:
params_fisheries = ikaf.readFisheriesXML(fisheries_xml_filepath)
params_fisheries.pop('P22')
params_fisheries.pop('P23')
params_fisheries.pop('L8')
params_fisheries.pop('S11')
_ = params_fisheries.pop('S14')

In [7]:
effort_filepath = "./../data/fisheries_effort/skj_fisheries_data_2021_Nov17.txt"
fisheries_df = fe.readFile(effort_filepath, header_position=0, columns_name=None)
fisheries_df = fe.removeEmptyEntries(fisheries_df,verbose=False)
# OPTIONAL :
# fisheries_df = fe.removeNoCatchEntries(fisheries_df,verbose=True)
# -------- #
fisheries_dict = fe.separateFisheries(fisheries_df)

# Make it faster to compute
fisheries_dict.pop(8)

# OPTIONAL :
# fisheries_dict = fe.predictEffortAllFisheries(fisheries_dict)
# -------- #
fisheries_dict = fe.rescaleFisheries(fisheries_dict)
fisheries_dict = fe.groupByFisheries(fisheries_dict)
fisheries_effort, _ = fe.fisheriesToDataSet(fisheries_dict)
fisheries_effort = fe.normalizeCoords(fisheries_effort, 30, 1)

In [8]:
fe.toTextFile(fisheries_dict)

In [9]:
import pandas as pd

In [12]:
pd.concat(fisheries_dict.values()).sort_values(['f','yr','mm','dd','lat','lon'])

,f,yr,mm,dd,lat,lon,gr,res,E,C
0,1,1950,1,15,21.5,141.5,P,1,1.00,4.010
1,1,1950,1,15,25.5,136.5,P,1,1.00,4.718
2,1,1950,1,15,26.5,140.5,P,1,11.00,0.000
3,1,1950,1,15,29.5,131.5,P,1,2.00,3.539
4,1,1950,1,15,34.5,139.5,P,1,9.00,0.000
...,...,...,...,...,...,...,...,...,...,...
49064,15,2014,8,15,13.5,212.5,P,1,0.12,1.200
49065,15,2014,8,15,13.5,213.5,P,1,0.12,1.200
49066,15,2014,8,15,13.5,214.5,P,1,0.12,1.200
49067,15,2014,8,15,13.5,215.5,P,1,0.12,1.200


In [8]:
convertion_tab = {'P1':1, 'P21':2, 'P3':3, 'S4':4, 'S5':5, 'S6':6, 'S7':7,
                  'L9':9, 'S10':10, 'S12':12, 'S13':13, 'P15':15}

In [9]:
fishing_mortality = ikaf.fishingMortality(fisheries_effort,
                                          params_fisheries,
                                          convertion_tab=convertion_tab)

---
# Affichage

In [10]:
taxis[0]

<xarray.DataArray 'Taxis_longitude_Feeding_Habitat_Cohort_0_to_36' (time: 384, lat: 60, lon: 101)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 64.5 62.5 60.5 58.5 56.5 ... -47.5 -49.5 -51.5 -53.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2010-12-01
  * lon      (lon) float32 89.5 91.5 93.5 95.5 97.5 ... 283.5 285.5 287.5 289.5
    cohorts  (time) int64 0 1 2 3 4 5 6 7 8 9 ... 36 36 36 36 36 36 36 36 36 36
Attributes:
    description:   Feeding habitat of a species that evolves over time.
    cohort_start:  0
    cohort_end:    36
    time_start:    0
    time_end:      383
    lat_min:       None
    lat_max:       None
    lon_min:       None
    lon_max:       None
    eF_list:       [3.999907116416026, 0.05, 0.897307278945509, 0.0, 0.0, 3.9...
    sigma_0:       1.492626509052
    sigma_K:       3.00000257690523
    T_star_1:      31.09996321152416
    T_star_K:      26.04960048814059
    bT:            2.83057972721579
    gamma:         0.0001000000005808978
    o_star:        3.764163854561827
    deltaT:        30.0
    space_reso:    2.0

In [11]:
diffusion

<xarray.DataArray 'K_Feeding_Habitat_Cohort_0_to_36' (time: 384, lat: 60, lon: 101)>
array([[[  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365],
        [  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365],
        [  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365],
        ...,
        [  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365],
        [  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365],
        [  103.22365,   103.22365,   103.22365, ...,   103.22365,
           103.22365,   103.22365]],

       [[  232.25322,   232.25322,   232.25322, ...,   232.25322,
           232.25322,   232.25322],
        [  232.25322,   232.25322,   232.25322, ...,   232.25322,
           232.25322,   232.25322],
        [  232.25322,   232.25322,   232.25322, ...,   232.25322,
           232.25322,   232.25322],
...
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ]],

       [[88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        ...,
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ],
        [88737.49   , 88737.49   , 88737.49   , ..., 88737.49   ,
         88737.49   , 88737.49   ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2010-12-01
  * lat      (lat) float32 64.5 62.5 60.5 58.5 56.5 ... -47.5 -49.5 -51.5 -53.5
  * lon      (lon) float32 89.5 91.5 93.5 95.5 97.5 ... 283.5 285.5 287.5 289.5
    cohorts  (time) int64 0 1 2 3 4 5 6 7 8 9 ... 36 36 36 36 36 36 36 36 36 36
Attributes:
    description:   Feeding habitat of a species that evolves over time.
    cohort_start:  0
    cohort_end:    36
    time_start:    0
    time_end:      383
    lat_min:       None
    lat_max:       None
    lon_min:       None
    lon_max:       None
    eF_list:       [3.999907116416026, 0.05, 0.897307278945509, 0.0, 0.0, 3.9...
    sigma_0:       1.492626509052
    sigma_K:       3.00000257690523
    T_star_1:      31.09996321152416
    T_star_K:      26.04960048814059
    bT:            2.83057972721579
    gamma:         0.0001000000005808978
    o_star:        3.764163854561827
    deltaT:        30.0
    space_reso:    2.0

In [12]:
fishing_mortality

<xarray.DataArray (time: 858, lat: 106, lon: 289)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2021-06-01
  * lat      (lat) float64 -49.5 -48.5 -47.5 -46.5 -45.5 ... 52.5 53.5 54.5 55.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 284.5 285.5 286.5 287.5 288.5
Attributes:
    Type:     Sum of E

---

In [13]:
slice_loc = fishing_mortality.loc["1979-01":"2010-12",-53.5:64.5,89.5:289.5]
print(slice_loc.sum().data)
slice_loc

78476.4295559435


<xarray.DataArray (time: 384, lat: 106, lon: 200)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2010-12-01
  * lat      (lat) float64 -49.5 -48.5 -47.5 -46.5 -45.5 ... 52.5 53.5 54.5 55.5
  * lon      (lon) float64 89.5 90.5 91.5 92.5 93.5 ... 285.5 286.5 287.5 288.5
Attributes:
    Type:     Sum of E

>- #### (1) If we don't normalize the feeding Habitat data structure variables the method (in `reindex_like()` function) must be `methode="nearest"`. Otherwise, dates will not match together and final data will be NaN values only.
>- #### (2) Large amount (3/4) of mortality is lost because space resolution is equal to 1 for mortality and equal to 2 for taxis. Lat and Lon coords in mortality field that don't match with taxis field will just be ignored.

In [14]:
slice_reindex_like = fishing_mortality.reindex_like(taxis[0])
print(slice_reindex_like.sum().data)
slice_reindex_like

17726.691643261423


<xarray.DataArray (time: 384, lat: 60, lon: 101)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * lat      (lat) float64 64.5 62.5 60.5 58.5 56.5 ... -47.5 -49.5 -51.5 -53.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2010-12-01
  * lon      (lon) float64 89.5 91.5 93.5 95.5 97.5 ... 283.5 285.5 287.5 289.5
Attributes:
    Type:     Sum of E